In [2]:
using DataFrames, CSV, DataArrays, JuMP, SCS

INFO: Recompiling stale cache file /mnt/juliabox/.julia/lib/v0.6/SCS.ji for module SCS.


In [4]:
GenData = readtable("gen_3.csv")
BusData = readtable("bus_3.csv")
BranchData = readtable("branch_3.csv")
Gen_cost = readtable("gencost_3.csv")

,x2,startup,shutdown,n,c2,c1,c0
1,2,1500,0,3,0.11,5.0,0
2,2,2000,0,3,0.085,1.2,0
3,2,3000,0,3,0.1225,1.0,0


In [5]:
# Function for the admittance matrix of the network

function ybus(BusData, BranchData)
    
    # Y = 1/Z
    BranchData[:z] = BranchData[:r] + im*BranchData[:x]
    BranchData[:y] = 1 ./BranchData[:z]
    
    # Numbers of lines and buses
    NLines = size(BranchData,1)
    NBuses = size(BusData,1)
    
    # Creating Admittance matrix Y
    Y = zeros(Complex, NBuses, NBuses)
    Ynm = zeros(Complex, NBuses, NBuses)
    
    B_lm = zeros(Complex, NBuses, NBuses)
    y_lm = zeros(Complex, NBuses, NBuses)
    
    for l in 1:NBuses
        B_lm[BranchData[l, :fbus], BranchData[l, :tbus]] = 0.5*im*BranchData[l, :b]
        B_lm[BranchData[l, :tbus], BranchData[l, :fbus]] = 0.5*im*BranchData[l, :b]
        y_lm[BranchData[l, :fbus], BranchData[l, :tbus]] = BranchData[l, :y]
        y_lm[BranchData[l, :tbus], BranchData[l, :fbus]] = BranchData[l, :y]
    end

    
       # Non-diagonal Y
    for l in 1:NLines
        Y[BranchData[l, :fbus], BranchData[l, :tbus]] = -BranchData[l, :y]
        Y[BranchData[l, :tbus], BranchData[l, :fbus]] = -BranchData[l, :y]
    end

    # Diagonal elements
    for i = 1:NBuses
        for j = 1:NLines
            if (i ==BranchData[j, :fbus])|| (i==BranchData[j, :tbus])
                Y[i,i] += BranchData[j, :y] + 0.5*im*BranchData[j, :b]
            end
        end
    end
    
    return Y, B_lm, y_lm
end

ybus (generic function with 1 method)

In [6]:
Ybus, B_lm, y_lm = ybus(BusData, BranchData)

(Complex[0.218996-2.32906im -0.0517392+1.1087im -0.167256+1.59537im; -0.0517392+1.1087im 0.0961343-1.94055im -0.0443951+1.33185im; -0.167256+1.59537im -0.0443951+1.33185im 0.211651-2.35222im], Complex[0+0im 0.0+0.15im 0.0+0.225im; 0.0+0.15im 0+0im 0.0+0.35im; 0.0+0.225im 0.0+0.35im 0+0im], Complex[0+0im 0.0517392-1.1087im 0.167256-1.59537im; 0.0517392-1.1087im 0+0im 0.0443951-1.33185im; 0.167256-1.59537im 0.0443951-1.33185im 0+0im])

In [7]:
function Ylm(B_lm, y_lm, l, m)
    e_l = zeros(NBuses,1)
    e_m = zeros(NBuses,1)
    e_l[l], e_m[m] = 1, 1
    
    Y_lm = (B_lm[l,m]+y_lm[l,m])*e_l*transpose(e_m)
end

Ylm (generic function with 1 method)

In [8]:
NBuses = size(BusData,1)

3

In [9]:
function Y_func(i, Ybus, NBuses)
    e_i = zeros(NBuses,1)
    e_i[i] = 1
    YY = e_i*transpose(e_i)*Ybus
    return YY
end
Y_func(1, Ybus, NBuses)

3×3 Array{Number,2}:
 0.218996-2.32906im  -0.0517392+1.1087im  -0.167256+1.59537im
      0.0+0.0im             0.0+0.0im           0.0+0.0im    
      0.0+0.0im             0.0+0.0im           0.0+0.0im    

In [10]:
Y_func_array = Matrix(NBuses,1)
for j in 1:NBuses
    Y_func_array[j] = Y_func(j, Ybus, NBuses)
end
Y_func_array[1] 

3×3 Array{Number,2}:
 0.218996-2.32906im  -0.0517392+1.1087im  -0.167256+1.59537im
      0.0+0.0im             0.0+0.0im           0.0+0.0im    
      0.0+0.0im             0.0+0.0im           0.0+0.0im    

In [11]:
function Y_i(i, YY, NBuses)
    e_i = zeros(NBuses,1)
    e_i[i] = 1
    
    Yi = 0.5 *[[real(YY + transpose(YY)) imag(transpose(YY) - YY)]; [imag(YY - transpose(YY)) real(YY + transpose(YY))]]
    
    Yi_i = -0.5 *[[imag(YY + transpose(YY)) real(YY - transpose(YY))]; [real(transpose(YY) - YY) imag(YY + transpose(YY))]]

    M = [[e_i*transpose(e_i) zeros(NBuses, NBuses)]; [zeros(NBuses, NBuses) e_i*transpose(e_i)]]
    
    return Yi, Yi_i, M
end

Y_i (generic function with 1 method)

In [12]:
Y_i_array = Matrix(NBuses,1)
for j in 1:NBuses
    Y_i_array[j] = Y_i(j, Y_func_array[j], NBuses)
end
size(Y_i_array)

(3, 1)

In [13]:
Y_k_array = Matrix(NBuses,1)
for j in 1:NBuses
    Y_k_array[j] = Y_i_array[j][1]
end
size(Y_k_array[1])

(6, 6)

In [14]:
Y_ii_array = Matrix(NBuses,1)
for j in 1:NBuses
    Y_ii_array[j] = Y_i_array[j][2]
end
Y_ii_array[1]

6×6 Array{Float64,2}:
  2.32906    -0.554348   -0.797684   -0.0         0.0258696   0.0836282
 -0.554348   -0.0        -0.0        -0.0258696  -0.0        -0.0      
 -0.797684   -0.0        -0.0        -0.0836282  -0.0        -0.0      
 -0.0        -0.0258696  -0.0836282   2.32906    -0.554348   -0.797684 
  0.0258696  -0.0        -0.0        -0.554348   -0.0        -0.0      
  0.0836282  -0.0        -0.0        -0.797684   -0.0        -0.0      

In [15]:
Y_M_array = Matrix(NBuses,1)
for j in 1:NBuses
    Y_M_array[j] = Y_i_array[j][3]
end
Y_M_array[1]

6×6 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

In [16]:
function YY_lm(B_lm, y_lm, l, m)
    Y_lm = Ylm(B_lm, y_lm, l, m)
    Y_lm_first = 0.5 *[[real(Y_lm + transpose(Y_lm)) imag(transpose(Y_lm) - Y_lm)]; [imag(Y_lm - transpose(Y_lm)) real(Y_lm + transpose(Y_lm))]]
    
    Y_lm_second = -0.5 *[[imag(Y_lm + transpose(Y_lm)) real(Y_lm - transpose(Y_lm))]; [real(transpose(Y_lm) - Y_lm) imag(Y_lm + transpose(Y_lm))]]
    
    return Y_lm_first, Y_lm_second
end

YY_lm (generic function with 1 method)

In [17]:
Y_lm_array = Matrix(NBuses,NBuses)
Y_lm_conj_array = Matrix(NBuses,NBuses)
for l in 1:NBuses
    for m in 1:NBuses
        if l != m
            Y_lm_array[l,m], Y_lm_conj_array[l, m] = YY_lm(B_lm, y_lm, l, m)
        else
            Y_lm_array[l,m], Y_lm_conj_array[l, m] = zeros(NBuses*2, NBuses*2), zeros(NBuses*2, NBuses*2)
        end
    end
end
Y_lm_array[1,2]

6×6 Array{Float64,2}:
 0.0         0.0258696  0.0   0.0        0.479348   0.0
 0.0258696   0.0        0.0  -0.479348   0.0        0.0
 0.0         0.0        0.0   0.0        0.0        0.0
 0.0        -0.479348   0.0   0.0        0.0258696  0.0
 0.479348    0.0        0.0   0.0258696  0.0        0.0
 0.0         0.0        0.0   0.0        0.0        0.0

In [ ]:
# Саша допиши тут, что нужно было

In [ ]:
# Solver 

In [18]:
# Define generators - it is a bus of type 2 or 3 (type 3 is a slack bus)
Gen_indices = Int64[]
for i in 1:NBuses
     if (BusData[i,:_type] == 2)||(BusData[i,:_type] == 3)
        push!(Gen_indices, BusData[i,:bus_i])
     end
end
NI =length(Gen_indices)

#NBuses=max(maximum(BranchData[:,:fbus]),maximum(BranchData[:,:tbus]));
NLines=size(BranchData,1);
NGen=NI;
Buses=1:NBuses;
Lines=1:NLines;
Gen=1:NGen;

In [19]:
# limints in branch powerflow
S_line = zeros(NBuses,NBuses)
S_line = [0 1000 1000;
          1000 0 60;
          1000 60 0;]
S_line[1,2]

1000

In [20]:
function AA_matrix(W,m, l, S_line, Y_lm_array, Y_lm_conj_array)
    AA = Matrix(3, 3)
    AA = [[-(S_line[m,l])^2 trace(Y_lm_array[l,m]*W) trace(Y_lm_conj_array[l,m]*W)];
                                                         [trace(Y_lm_array[l,m]*W) -1 0];
                                                        [trace(Y_lm_conj_array[l,m]*W) 0 -1]]
    return AA
end

AA_matrix (generic function with 1 method)

In [36]:
function BB_matrix(W, i, Y_k_array, BusData, Gen_cost)
    return [Gen_cost[i, :c2]*(trace(Y_k_array[i]*W)+BusData[i, :Pd])^2 -sqrt(Gen_cost[i, :c2])*(trace(Y_k_array[i]*W)+BusData[i, :Pd]);
                                      -sqrt(Gen_cost[i, :c2])*(trace(Y_k_array[i]*W)+BusData[i, :Pd]) 1;]
    
end

BB_matrix (generic function with 1 method)

In [29]:
AA_matrix(zz,1, 2, S_line, Y_lm_array, Y_lm_conj_array)

3×3 Array{Float64,2}:
 -1.0e6   0.0   0.0
  0.0    -1.0   0.0
  0.0     0.0  -1.0

In [22]:
mm = Model(solver = SCSSolver())
@variable(mm, W[1:6, 1:6], SDP)
#@variable(mm, AA[1:3, 1:3], SDP)
#@variable(mm, BB[1:2, 1:2], SDP)

constr = Array{JuMP.ConstraintRef, 1}()

for l = 1:NBuses,  m = 1:NBuses 
    @expression(mm, expr, [[-(S_line[m,l])^2 trace(Y_lm_array[l,m]*W) trace(Y_lm_conj_array[l,m]*W)];
                                                         [trace(Y_lm_array[l,m]*W) -1 0];
                                                         [trace(Y_lm_conj_array[l,m]*W) 0 -1]])
    push!(constr, @SDconstraint(mm, expr >= zeros(3,3)))
end

constr1 = Array{JuMP.ConstraintRef, 1}()

for i = 1:NBuses
    @expression(mm, expr1, [Gen_cost[i, :c2]*(trace(Y_k_array[i]*W)+BusData[i, :Pd])^2 -sqrt(Gen_cost[i, :c2])*(trace(Y_k_array[i]*W)+BusData[i, :Pd]);
                                      -sqrt(Gen_cost[i, :c2])*(trace(Y_k_array[i]*W)+BusData[i, :Pd]) 1;])
    push!(constr1, @SDconstraint(mm, expr1 >= zeros(2,2)))
end
                
# Nodal balance equation 9b
@constraint(mm, BalanceEq[i = 1:NI], GenData[i, :Pmin] - BusData[i, :Pd] <= trace(Y_k_array[i]*W) <= GenData[i, :Pmax] - BusData[i, :Pd])
# Nodal balance equation 9c
@constraint(mm, BalanceEqq[i = 1:NI], GenData[i, :Qmin] - BusData[i, :Qd] <= trace(Y_ii_array[i]*W) <= GenData[i, :Qmax] - BusData[i, :Qd])
# Nodal balance equation 9d
@constraint(mm, BalanceEqqq[i = 1:NBuses], BusData[i, :Vmin]^2 <= trace(Y_M_array[i]*W) <= BusData[i, :Vmax]^2)

@objective(mm, Min, sum(Gen_cost[i, :c2]*(trace(Y_k_array[i]*W) + BusData[i, :Pd])^2 + Gen_cost[i, :c1]*(trace(Y_k_array[i]*W) + BusData[i, :Pd]) + Gen_cost[i, :c0] for i in Gen_indices))
                
                
solve(mm)

LoadError: [91mJuMP does not support quadratic objectives for conic problems[39m

In [ ]:
 [[Gen_cost[i, :c2]*(trace(Y_k_array[i]*W)+BusData[i, :Pd])^2 -sqrt(Gen_cost[i, :c2])*(trace(Y_k_array[i]*W)+BusData[i, :Pd]);
                                      [-sqrt(Gen_cost[i, :c2])*(trace(Y_k_array[i]*W)+BusData[i, :Pd]) 1]

In [ ]:
# Nodal balance equation 9b
@constraint(mm, BalanceEq[i = 1:NI], GenData[i, :Pmin] - BusData[i, :Pd] <= trace(Y_k_array[i]*W) <= GenData[i, :Pmax] - BusData[i, :Pd])
# Nodal balance equation 9c
@constraint(mm, BalanceEqq[i = 1:NI], GenData[i, :Qmin] - BusData[i, :Qd] <= trace(Y_ii_array[i]*W) <= GenData[i, :Qmax] - BusData[i, :Qd])
# Nodal balance equation 9d
@constraint(mm, BalanceEqqq[i = 1:NBuses], BusData[i, :Vmin]^2 <= trace(Y_M_array[i]*W) <= BusData[i, :Vmax]^2)

@objective(mm, Min, sum(Gen_cost[i, :c2]*(trace(Y_k_array[i]*W) + BusData[i, :Pd])^2 + Gen_cost[i, :c1]*(trace(Y_k_array[i]*W) + BusData[i, :Pd]) + Gen_cost[i, :c0] for i in Gen_indices))
                

In [ ]:
@SDconstraint(mm, FlowLimits[l = 1:NBuses, m = 1:NBuses], AA_matrix(W,m, l, S_line, Y_lm_array, Y_lm_conj_array) >= zeros(3,3))


In [ ]:
# Дальше пока Саня

In [31]:
BranchData

,fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax,z,y
1,1,3,0.065,0.62,0.45,250,250,250,0,0,1,-360,360,0.065+0.62im,0.167256-1.59537im
2,3,2,0.025,0.75,0.7,250,250,250,0,0,1,-360,360,0.025+0.75im,0.0443951-1.33185im
3,1,2,0.042,0.9,0.3,150,150,150,0,0,1,-360,360,0.042+0.9im,0.0517392-1.1087im


In [26]:
expr = @expression(mm, [l = 1:NBuses, m = 1:NBuses], [[-(S_line[m,l])^2 trace(Y_lm_array[l,m]*W) trace(Y_lm_conj_array[l,m]*W)];
                                                         [trace(Y_lm_array[l,m]*W) -1 0];
                                                         [trace(Y_lm_conj_array[l,m]*W) 0 -1]])

LoadError: [91mCollection of expressions with @expression must be linear. For quadratic expressions, use your own array.[39m

In [48]:
# Nodal balance equation 9b
@constraint(mm, BalanceEq[i = 1:NI], GenData[i, :Pmin] - BusData[i, :Pd] <= trace(Y_k_array[i]*W) <= GenData[i, :Pmax] - BusData[i, :Pd])
# Nodal balance equation 9c
@constraint(mm, BalanceEqq[i = 1:NI], GenData[i, :Qmin] - BusData[i, :Qd] <= trace(Y_ii_array[i]*W) <= GenData[i, :Qmax] - BusData[i, :Qd])
# Nodal balance equation 9d
@constraint(mm, BalanceEqqq[i = 1:NBuses], BusData[i, :Vmin]^2 <= trace(Y_M_array[i]*W) <= BusData[i, :Vmax]^2)
# Line limits
@constraint(mm, FlowLimits[l = 1:NBuses, m = 1:NBuses], AA .== zeros(3,3))

LoadError: [91mMethodError: Cannot `convert` an object of type Array{JuMP.ConstraintRef{JuMP.Model,JuMP.GenericRangeConstraint{JuMP.GenericAffExpr{Float64,JuMP.Variable}}},2} to an object of type JuMP.ConstraintRef
This may have arisen from a call to the constructor JuMP.ConstraintRef(...),
since type constructors fall back to convert methods.[39m

In [38]:
[ "s", 1]

2-element Array{Any,1}:
  "s"
 1   

In [ ]:
mm = Model(solver = IpoptSolver())
@variable(mm, W[1:6, 1:6], SDP)
#@variable(mm, AA[1:3, 1:3], SDP)
#@variable(mm, BB[1:2, 1:2], SDP)

constr = Array{JuMP.ConstraintRef, 1}()

for l = 1:NBuses,  m = 1:NBuses 
    @expression(mm, expr, [AA[i,j] BB[i,j]; AA[i,j] BB[i,j]])
    push!(constr, @SDconstraint(mm, expr >= zeros(2,2)))
end

In [ ]:
@constraints(m, FlowLimits[l = 1:NLines, m = 1:NLines], begin
    AA[1,1] == -(S_line[m,l])^2
    AA[1,2] == trace(Y_lm_array[l,m]*W)
    AA[1,3] == trace(Y_lm_conj_array[l,m]*W)
    AA[2,1] == trace(Y_lm_array[l,m]*W)
    AA[2,2] == -1
    AA[2,3] == 0
    AA[3,1] == trace(Y_lm_conj_array[l,m]*W)
    AA[3,2] == 0
    AA[3,3] == -1
end)

In [ ]:
@constraint(mm, FlowLimits[l = 1:NLines, m = 1:NLines],  AA == [[-(S_line[m,l])^2 trace(Y_lm_array[l,m]*W) trace(Y_lm_conj_array[l,m]*W)];
                                                         [trace(Y_lm_array[l,m]*W) -1 0];
                                                         [trace(Y_lm_conj_array[l,m]*W) 0 -1]])
# Generation
@constraint(mm, BalanceEqqqq[i = 1:NI],  BB  == [[Gen_cost[i, :c2]*(trace(Y_k_array[i]*W)+BusData[i, :Pd])^2 -sqrt(Gen_cost[i, :c2])*(trace(Y_k_array[i]*W)+BusData[i, :Pd]);
                                      [-sqrt(Gen_cost[i, :c2])*(trace(Y_k_array[i]*W)+BusData[i, :Pd]) 1])

In [ ]:
@addConstraints(m, FlowLimits[l = 1:NLines, m = 1:NLines], begin
    AA[1,1] == -(S_line[m,l])^2
    AA[1,2] == trace(Y_lm_array[l,m]*W)
    AA[1,3] == trace(Y_lm_conj_array[l,m]*W)
    AA[2,1] == trace(Y_lm_array[l,m]*W)
    AA[2,2] == -1
    AA[2,3] == 0
    AA[3,1] == trace(Y_lm_conj_array[l,m]*W)
    AA[3,2] == 0
    AA[3,3] == -1
end)

In [69]:
@objective(mm, Min, sum(Gen_cost[i, :c2]*(trace(Y_k_array[i]*W) + BusData[i, :Pd])^2 + Gen_cost[i, :c1]*(trace(Y_k_array[i]*W) + BusData[i, :Pd]) + Gen_cost[i, :c0] for i in Gen_indices))

0.005275494538845252 W[1,1]² - 0.0024927425683582295 W[1,1]*W[1,2] + 0.0005220037433709549 W[1,2]² - 0.008058246509332274 W[1,1]*W[1,3] + 0.0019038152681448443 W[1,2]*W[1,3] + 0.006504114834436714 W[1,3]² - 0.053415912179104914 W[1,1]*W[1,5] + 0.002868152436104148 W[1,2]*W[1,5] + 0.04079604146024667 W[1,3]*W[1,5] + 0.2396955964458467 W[1,5]² - 0.07686327439670784 W[1,1]*W[1,6] + 0.01815946871153544 W[1,2]*W[1,6] - 0.006670887009678673 W[1,3]*W[1,6] + 0.38913147239004514 W[1,5]*W[1,6] + 0.5917589922739582 W[1,6]² - 0.0008455645293615056 W[1,2]*W[2,2] + 0.018119239914889403 W[1,5]*W[2,2] + 0.0007855531777469824 W[2,2]² + 0.0003904843428329807 W[1,2]*W[2,3] + 0.0018192144891723537 W[1,3]*W[2,3] - 0.0083675216321353 W[1,5]*W[2,3] - 0.01735250743518245 W[1,6]*W[2,3] - 0.0007255418261324593 W[2,2]*W[2,3] + 0.0004089672222625988 W[2,3]² + 0.053415912179104914 W[1,1]*W[2,4] - 0.002868152436104148 W[1,2]*W[2,4] - 0.04079604146024667 W[1,3]*W[2,4] - 0.4793911928916934 W[1,5]*W[2,4] - 0.389131472

In [70]:
solve(mm)

LoadError: [91mJuMP does not support quadratic objectives for conic problems[39m

In [ ]:
scopf("bus_9.csv", "branch_9.csv", "gen_9.csv", "gencost_9.csv", "Corrective")